In [1]:
import pandas as pd
import janitor

In [2]:
# Group and keep columns with 'as_index'
# https://stackoverflow.com/questions/31569549/how-to-groupby-a-dataframe-in-pandas-and-keep-columns

### Daily report data

In [3]:
data = pd.read_csv('../latest_raw.csv', encoding='utf-8')
data = data.drop(['MUNICIPIO_RES'], axis=1) # readily drop this column

In [4]:
print(data.set_index(['ID_REGISTRO']).index.is_unique)

False


In [5]:
data['FECHA_INGRESO'] = pd.to_datetime(data.FECHA_INGRESO, format='%Y-%m-%d', errors='coerce')
data['FECHA_INGRESO'] = data['FECHA_INGRESO'].dt.strftime('%Y-%m-%d')
data = data.set_index(pd.DatetimeIndex(data['FECHA_INGRESO'])).sort_index()

In [6]:
len(data)

765794

### Note
Daily report for april 20 (ie, reported in 21 april) stops using testing site (ENTIDAD_UM),
<br>
And starts using region of residence (ENTIDAD_RES) so we shall treat both sets of data accordingly

In [7]:
dat = data.copy()

In [8]:
tbl_x = dat.filter_date('FECHA_INGRESO','2020-01-06','2020-04-20')
tbl_x = tbl_x.drop(['ENTIDAD_RES'], axis=1) # drop by region
tbl_x = tbl_x.set_index(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_UM', 'ID_REGISTRO', 'PAIS_ORIGEN', 'RESULTADO']
)
tbl_x = tbl_x.groupby(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_UM', 'ID_REGISTRO', 'PAIS_ORIGEN', 'RESULTADO'], as_index=True).count().reset_index()
tbl_x = tbl_x.rename(columns={'ENTIDAD_UM':'Region_ID'})

In [9]:
tbl_y = dat.filter_date('FECHA_INGRESO','2020-04-21','2022-12-31') # the long con
tbl_y = tbl_y.drop(['ENTIDAD_UM'], axis=1) # drop by region
tbl_y = tbl_y.set_index(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_RES', 'ID_REGISTRO', 'PAIS_ORIGEN', 'RESULTADO']
)
tbl_y = tbl_y.groupby(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_RES', 'ID_REGISTRO', 'PAIS_ORIGEN', 'RESULTADO'], as_index=True).count().reset_index()
tbl_y = tbl_y.rename(columns={'ENTIDAD_RES':'Region_ID'})

In [10]:
tbl = pd.concat([tbl_x, tbl_y])

In [11]:
tbl

,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,Region_ID,ID_REGISTRO,PAIS_ORIGEN,RESULTADO
0,2020-01-06,2020-01-02,9999-99-99,14,05afcc,Local,2
1,2020-01-06,2020-01-02,9999-99-99,23,1c405f,Local,2
2,2020-01-06,2020-01-03,9999-99-99,5,0131f0,Local,2
3,2020-01-06,2020-01-03,9999-99-99,9,105f1c,Local,2
4,2020-01-06,2020-01-03,9999-99-99,14,01fefc,Local,2
...,...,...,...,...,...,...,...
68148,2020-05-04,2020-05-04,9999-99-99,28,19e963,Local,3
68149,2020-05-04,2020-05-04,9999-99-99,28,1a4556,Local,3
68150,2020-05-04,2020-05-04,9999-99-99,28,1d4827,Local,3
68151,2020-05-04,2020-05-04,9999-99-99,28,1dc515,Local,3


In [12]:
tbl = tbl.rename(
    columns={
        'FECHA_INGRESO':'Date_Confirmed',
        'FECHA_SINTOMAS':'Date_Symptoms',
        'FECHA_DEF':'Date_Death',
        'ID_REGISTRO':'Case_ID',
        'RESULTADO':'Status',
        'PAIS_ORIGEN':'Origin'
    })

In [13]:
tbl

,Date_Confirmed,Date_Symptoms,Date_Death,Region_ID,Case_ID,Origin,Status
0,2020-01-06,2020-01-02,9999-99-99,14,05afcc,Local,2
1,2020-01-06,2020-01-02,9999-99-99,23,1c405f,Local,2
2,2020-01-06,2020-01-03,9999-99-99,5,0131f0,Local,2
3,2020-01-06,2020-01-03,9999-99-99,9,105f1c,Local,2
4,2020-01-06,2020-01-03,9999-99-99,14,01fefc,Local,2
...,...,...,...,...,...,...,...
68148,2020-05-04,2020-05-04,9999-99-99,28,19e963,Local,3
68149,2020-05-04,2020-05-04,9999-99-99,28,1a4556,Local,3
68150,2020-05-04,2020-05-04,9999-99-99,28,1d4827,Local,3
68151,2020-05-04,2020-05-04,9999-99-99,28,1dc515,Local,3


In [14]:
tbl

,Date_Confirmed,Date_Symptoms,Date_Death,Region_ID,Case_ID,Origin,Status
0,2020-01-06,2020-01-02,9999-99-99,14,05afcc,Local,2
1,2020-01-06,2020-01-02,9999-99-99,23,1c405f,Local,2
2,2020-01-06,2020-01-03,9999-99-99,5,0131f0,Local,2
3,2020-01-06,2020-01-03,9999-99-99,9,105f1c,Local,2
4,2020-01-06,2020-01-03,9999-99-99,14,01fefc,Local,2
...,...,...,...,...,...,...,...
68148,2020-05-04,2020-05-04,9999-99-99,28,19e963,Local,3
68149,2020-05-04,2020-05-04,9999-99-99,28,1a4556,Local,3
68150,2020-05-04,2020-05-04,9999-99-99,28,1d4827,Local,3
68151,2020-05-04,2020-05-04,9999-99-99,28,1dc515,Local,3


In [15]:
geo = pd.read_csv('../data/geo/entidades.csv')

In [16]:
geo.keys()

Index(['CLAVE_ENTIDAD', 'ENTIDAD_FEDERATIVA', 'ABREVIATURA'], dtype='object')

In [17]:
geo = geo.rename(columns={'CLAVE_ENTIDAD':'Region_ID', 'ENTIDAD_FEDERATIVA':'Region', 'ABREVIATURA':'Region_Key'})

In [47]:
output = pd.merge(tbl, geo, on='Region_ID') 

In [48]:
output = output[['Date_Confirmed', 'Date_Symptoms', 'Date_Death', 'Region', 'Region_ID', 'Region_Key', 'Origin', 'Status']]

In [49]:
output

,Date_Confirmed,Date_Symptoms,Date_Death,Region,Region_ID,Region_Key,Origin,Status
0,2020-01-06,2020-01-02,9999-99-99,JALISCO,14,JC,Local,2
1,2020-01-06,2020-01-03,9999-99-99,JALISCO,14,JC,Local,2
2,2020-01-06,2020-01-06,9999-99-99,JALISCO,14,JC,Local,2
3,2020-01-07,2020-01-01,9999-99-99,JALISCO,14,JC,Local,2
4,2020-01-07,2020-01-02,9999-99-99,JALISCO,14,JC,Local,2
...,...,...,...,...,...,...,...,...
127141,2020-05-03,2020-04-30,9999-99-99,COLIMA,6,CM,Local,3
127142,2020-05-03,2020-05-02,9999-99-99,COLIMA,6,CM,Local,3
127143,2020-05-03,2020-05-03,9999-99-99,COLIMA,6,CM,Local,3
127144,2020-05-04,2020-04-30,9999-99-99,COLIMA,6,CM,Local,3


In [50]:
output['Region'] = output['Region'].str.replace('COAHUILA DE ZARAGOZA','COAHUILA')
output['Region'] = output['Region'].str.replace('MICHOACÁN DE OCAMPO','MICHOACAN')
output['Region'] = output['Region'].str.replace('VERACRUZ DE IGNACIO DE LA LLAVE','VERACRUZ')

In [51]:
output = output.sort_values(['Date_Confirmed', 'Date_Symptoms', 'Region'])

In [52]:
output.Region.unique()

array(['JALISCO', 'QUINTANA ROO', 'CIUDAD DE MÉXICO', 'COAHUILA',
       'VERACRUZ', 'BAJA CALIFORNIA', 'DURANGO', 'TABASCO', 'MÉXICO',
       'NAYARIT', 'SONORA', 'GUANAJUATO', 'MORELOS', 'TAMAULIPAS',
       'HIDALGO', 'PUEBLA', 'AGUASCALIENTES', 'BAJA CALIFORNIA SUR',
       'YUCATÁN', 'MICHOACAN', 'GUERRERO', 'OAXACA', 'SINALOA',
       'NUEVO LEÓN', 'TLAXCALA', 'CHIHUAHUA', 'ZACATECAS',
       'SAN LUIS POTOSÍ', 'CHIAPAS', 'QUERÉTARO', 'CAMPECHE', 'COLIMA'],
      dtype=object)

In [53]:
output.Origin.unique()

array(['Local', 'Alemania', 'Cuba', 'Grecia', 'Estados Unidos de Am�rica',
       'Espa�a', 'Colombia', 'Egipto', 'El Salvador', 'Ecuador',
       'Venezuela', 'Hait�', 'Rep�blica de Honduras', 'Guatemala', 'Otro',
       'Bolivia', 'Nicaragua', 'China', 'Per�', 'Camer�n', 'Chile',
       'Jap�n', 'Eritrea', 'Canad�', 'Brasil', 'Belice', 'Irlanda',
       'Costa de Marfil', 'Italia', 'Francia'], dtype=object)

In [54]:
output

,Date_Confirmed,Date_Symptoms,Date_Death,Region,Region_ID,Region_Key,Origin,Status
0,2020-01-06,2020-01-02,9999-99-99,JALISCO,14,JC,Local,2
7945,2020-01-06,2020-01-02,9999-99-99,QUINTANA ROO,23,QR,Local,2
15105,2020-01-06,2020-01-03,9999-99-99,CIUDAD DE MÉXICO,9,DF,Local,2
10228,2020-01-06,2020-01-03,9999-99-99,COAHUILA,5,CL,Local,2
1,2020-01-06,2020-01-03,9999-99-99,JALISCO,14,JC,Local,2
...,...,...,...,...,...,...,...,...
88254,2020-05-04,2020-05-04,9999-99-99,TAMAULIPAS,28,TS,Local,3
88255,2020-05-04,2020-05-04,9999-99-99,TAMAULIPAS,28,TS,Local,3
88256,2020-05-04,2020-05-04,9999-99-99,TAMAULIPAS,28,TS,Local,3
88257,2020-05-04,2020-05-04,9999-99-99,TAMAULIPAS,28,TS,Local,3


### By status

In [239]:
# Active cases

In [244]:
tbl_1 = output.copy()
tbl_1 = tbl_1[tbl_1.Date_Death.str.startswith(('9999'))] # active cases in patients not deceased

In [245]:
positive = tbl_1[tbl_1['Status']==1]
positive = positive.drop(['Date_Death'], axis=1)

In [246]:
negative = tbl_1[tbl_1['Status']==2]
negative = negative.drop(['Date_Death'], axis=1)

In [247]:
suspects = tbl_1[tbl_1['Status']==3]
suspects = suspects.drop(['Date_Death'], axis=1)

In [248]:
# Deceased patients

In [249]:
tbl_2 = tbl[~tbl.Date_Death.str.startswith(('9999'))] # to-do: check if 
deceased = tbl_2

In [250]:
len(deceased)

3932

In [252]:
# Save

In [253]:
positive.to_csv('../total_positive.csv', index=False)

In [254]:
negative.to_csv('../total_negative.csv', index=False)

In [255]:
suspects.to_csv('../total_suspects.csv', index=False)

In [256]:
deceased.to_csv('../total_deceased.csv', index=False)

In [257]:
# Periodicity on reports

In [267]:
test1 = positive.reset_index()
test1.Date_Confirmed = test1.Date_Confirmed.astype('str')

In [268]:
test1 = test1[test1.Date_Confirmed.str.startswith(('2020-05-01'))]

In [269]:
len(test1)

376

In [270]:
test2 = suspects.reset_index()
test2.Date_Confirmed = test2.Date_Confirmed.astype('str')

In [271]:
test2 = test2[test2.Date_Confirmed.str.startswith(('2020-05-01'))]

In [272]:
len(test2)

2636

In [ ]:
# Latest time series

In [ ]:
latest = output.copy()

In [265]:
latest = latest[latest.Date_Death.str.startswith(('9999'))] # active cases in patients not deceased
latest = latest[latest['Status']==1]
latest = latest[['Date_Confirmed','Date_Symptoms','Region','Region_ID','Region_Key','Origin','Status']]

In [266]:
latest.to_csv('../latest.csv', index=False)